## Importing libraries

In [1]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import requests
import math
import json

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Getting the api url

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'S(GLE BSLEDKT/RQNL AAS)GMC ONATAE', 'calculationPrice': 'close', 'open': 122.14, 'openTime': 1658465449743, 'openSource': 'iffcloai', 'close': 125.15, 'closeTime': 1618525554221, 'closeSource': 'ioacilff', 'high': 122.73, 'highTime': 1687872619254, 'highSource': 'leai ycd  n5etier1edmpu', 'low': 124.01, 'lowTime': 1648555148141, 'lowSource': ' te dul5yedenimpc iea1r', 'latestPrice': 125.79, 'latestSource': 'Close', 'latestTime': 'March 12, 2021', 'latestUpdate': 1685359706577, 'latestVolume': 88469180, 'iexRealtimePrice': 125.79, 'iexRealtimeSize': 417, 'iexLastUpdated': 1625053261718, 'delayedPrice': 126.13, 'delayedPriceTime': 1694993543477, 'oddLotDelayedPrice': 126, 'oddLotDelayedPriceTime': 1672469856223, 'extendedPrice': 123.18, 'extendedChange': 0.05, 'extendedChangePercent': 0.00043, 'extendedPriceTime': 1617111536095, 'previousClose': 126, 'previousVolume': 106617068, 'change': -0.94, 'changePercent': -0.00765,

In [4]:
price = data['latestPrice']
market_cap = data['marketCap']
market_cap

2087564237495

## Getting all of the stock data

In [5]:
columns_list = ['Ticker', 'Stock Price', 'Makret Cap', 'No. of shares to buy']
df = pd.DataFrame(columns=columns_list)
df

,Ticker,Stock Price,Makret Cap,No. of shares to buy


In [6]:
df.append(pd.Series(data=[symbol, price, market_cap, 'na'], index=columns_list), ignore_index=True)

,Ticker,Stock Price,Makret Cap,No. of shares to buy
0,AAPL,125.79,2087564237495,na


for symbol in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    
    price = data['latestPrice']
    market_cap = data['marketCap']
    df = df.append(pd.Series(data=[symbol, price, market_cap, 'na'], index=columns_list), ignore_index=True)

df.head()

## Improve the speed of code

In [7]:
# define a function that splits the symbol list into several sublists
def chunks(list, n):
    for i in range(0, len(list), n):
        yield list[i:i + n]

In [8]:
sublists = list(chunks(stocks['Ticker'], 100))
sublists_string = []

for i in range(0, len(sublists)):
    sublists_string.append(','.join(sublists[i]))

sublists_string

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD',
 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,I

In [9]:
new_df = pd.DataFrame(columns=columns_list)

for sub_string in sublists_string:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={sub_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'

    new_data = requests.get(batch_api_call_url).json()

    for symbol in sub_string.split(','):
        price = new_data[symbol]['quote']['latestPrice']
        market_cap = new_data[symbol]['quote']['marketCap']
        new_df = new_df.append(pd.Series(data=[symbol, price, market_cap, 'na'], index=columns_list), ignore_index=True)

new_df

,Ticker,Stock Price,Makret Cap,No. of shares to buy
0,A,125.09,37890055606,na
1,AAL,24.48,15475527069,na
2,AAP,179.97,12240618460,na
3,AAPL,126.63,2078242928527,na
4,ABBV,111.26,195555364533,na
...,...,...,...,...
500,YUM,109.87,32498983724,na
501,ZBH,168.11,34539702488,na
502,ZBRA,480.49,25514731823,na
503,ZION,61.11,9710855816,na


## Calculate the number of shares to buy

In [10]:
# specify the size of the portfolio
portfolio = input('Enter the size of your portfolio: ')

try:
    val = float(portfolio)
except ValueError:
    print("That's not a number! Please try again.\n")
    portfolio = input('Enter the size of your portfolio: ')
    val = float(portfolio)

Enter the size of your portfolio:  1000000


In [11]:
position_size = val / len(new_df.index)
position_size

1980.1980198019803

In [12]:
# assuming that fractional shares are not allowed
new_df['No. of shares to buy'] = np.floor(position_size / new_df['Stock Price'])
new_df

,Ticker,Stock Price,Makret Cap,No. of shares to buy
0,A,125.09,37890055606,15.0
1,AAL,24.48,15475527069,80.0
2,AAP,179.97,12240618460,11.0
3,AAPL,126.63,2078242928527,15.0
4,ABBV,111.26,195555364533,17.0
...,...,...,...,...
500,YUM,109.87,32498983724,18.0
501,ZBH,168.11,34539702488,11.0
502,ZBRA,480.49,25514731823,4.0
503,ZION,61.11,9710855816,32.0


## Store the dataframe into an excel file

In [13]:
import xlsxwriter

In [14]:
writer = pd.ExcelWriter('Recommended_trades.xlsx', engine='xlsxwriter')
new_df.to_excel(writer, 'Recommended Trades', index=False)

### Formatting

In [15]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
writer.save()

In [16]:
columns_format = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalisation', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in columns_format:
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, columns_format[column][1])
writer.save()